# Configuration

Different variables can be defined here (e.g., paths of data). These changes will be saved in a configuration file (by default, ```config.ini```) and will therefore not have to be redefined each time. Note that other tabs can also modified this file.

In [1]:
import ipywidgets as widgets
import os

list_config_files = [x for x in os.listdir() if '.ini' in x]

# config_w = widgets.Text(value=(list_config_files[0] if list_config_files != [] else 'config.ini'),
#                         description='Configuration file:', disabled=False)

config_w = widgets.Combobox(value=(list_config_files[0] if list_config_files != [] else 'config.ini'),
    placeholder='Choose config file',
    options=list_config_files,
    description='Configuration file:',
    ensure_option=True,
    disabled=False
)
widgets.HBox([config_w])

The database is a structure of files organized in different folders. Some files are mandatory and other are optionnals. Let first introduce the mandatory files and associated options (the next section for optional files and options).

All paths are given according to the root of the database, the default minimum structure is:
- a ```.\data\``` folder that contains the files of data (one per patient).
- a ```.\*.xlsx``` file that contain the metadata of patients.

In [2]:
from datetime import datetime
from functions.config import Config

config_filename = config_w.value

if not os.path.isfile(config_filename):
    f = open(config_filename, "w")
    f.close()

config = Config(config_filename)
config.read()

path_database = widgets.Text(value=(config.get_value('database') if config.is_key('database') else ''),
                             description='Path of database:', disabled=False)

metadata_filename = widgets.Text(value=(config.get_value('metadata') if config.is_key('metadata') else ''),
                                 description='Metadata file:', disabled=False)

def on_button_save_clicked(var):
    config.set_value(path_database.value, 'database', section='PATH')
    config.set_value(metadata_filename.value, 'metadata', section='PATH')
    config.write()

button_save = widgets.Button(description='Save "{}"'.format(config_filename))
button_save.on_click(on_button_save_clicked)

widgets.VBox([path_database, 
              widgets.HBox([metadata_filename, widgets.Label(value='This can be a relative path from the path of the database (previous field).')]),
              button_save])

The next options are used for specific analysis. Some of these options allow to go further in the analyses and others to modify the analysis framework (e.g., the cut-off date allows to repeat analyses at a given time).

In [4]:
from datetime import datetime
from functions.config import Config

config_filename = config_w.value

if not os.path.isfile(config_filename):
    f = open(config_filename, "w")
    f.close()

config = Config(config_filename)
config.read()

visits = widgets.Text(value=(str(config.get_value('visits')).replace("'", "")[1:-1] if config.is_key('visits') else ''), placeholder='Visits', description='Visits:',
                      disabled=False)


cutoff_date = widgets.DatePicker(description='Cut-off date:', value=(datetime.strptime(config.get_value('cutoff_date'), '%Y-%m-%d') if config.is_key('cutoff_date') else None), 
                                 disabled=False)

followup = widgets.IntText(value=(config.get_value('followup') if config.is_key('followup') else 0),
                description='Follow-up:', disabled=False)

def on_button_save_clicked(var):
    config.set_value(visits.value.replace(' ','').split(','), 'visits', section='OTHER')
    if cutoff_date.value is not None:
        config.set_value(str(cutoff_date.value), 'cutoff_date', section='OTHER')
    else:
        config.remove_value('cutoff_date', section='OTHER')
    config.set_value(followup.value, 'followup', section='OTHER')
    config.write()

button_save = widgets.Button(description='Save "{}"'.format(config_filename))
button_save.on_click(on_button_save_clicked)

widgets.VBox([cutoff_date,
              widgets.HBox([followup, widgets.Label(value='Duration in days')]),
              widgets.HBox([visits, widgets.Label(value='List of visits separeted by commas. Format is unit letter and number (e.g., "W5" for 5 weeks or "M2" for 2 months)')]),
              button_save])

TypeError: option values must be strings